## Comment Toxicity


In [23]:
!pip install tensorflow==2.15 pandas numpy scikit-learn

In [24]:
pip show tensorflow

Name: tensorflow
Version: 2.15.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine_rl, tensorflow-text, tensorflow_decision_forests, tf_keras


In [3]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization, Embedding, Bidirectional, LSTM, Dense, Dropout
import pickle
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score, confusion_matrix
from tensorflow.keras.metrics import Precision, Recall


In [4]:
tf.keras.mixed_precision.set_global_policy('mixed_float16')

In [ ]:
MAX_FEATURES = 100000
OUTPUT_SEQUENCE_LENGTH = 1800
EMBEDDING_DIM = 64
LSTM_UNITS = 32
BATCH_SIZE = 128
VALIDATION_SPLIT = 0.2
POSITIVE_WEIGHT_SCALE = 2.0
PREDICTION_THRESHOLD = 0.3
OVERSAMPLE_FACTOR = 3

In [10]:
df = pd.read_csv("/content/drive/MyDrive/Data/Comment Toxicity/train.csv")
comments = df['comment_text'].values
labels = df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values

In [11]:
positive_indices = np.where(labels.sum(axis=1) > 0)[0]
train_comments = np.concatenate([comments] + [comments[positive_indices]] * (OVERSAMPLE_FACTOR - 1))  # Changed: Oversample 3x
train_labels = np.concatenate([labels] + [labels[positive_indices]] * (OVERSAMPLE_FACTOR - 1))
indices = np.random.permutation(len(train_comments))
train_comments, train_labels = train_comments[indices], train_labels[indices]

In [12]:
num_samples = len(comments)
num_train = int((1 - VALIDATION_SPLIT) * num_samples)
indices = np.random.permutation(num_samples)
train_indices, val_indices = indices[:num_train], indices[num_train:]

train_comments, val_comments = comments[train_indices], comments[val_indices]
train_labels, val_labels = labels[train_indices], labels[val_indices]

In [13]:
print(f"Total samples: {num_samples}")
print(f"Training samples: {len(train_comments)}, Steps per epoch: {len(train_comments) / BATCH_SIZE}")
print(f"Validation samples: {len(val_comments)}")

Total samples: 159568
Training samples: 127654, Steps per epoch: 997.296875
Validation samples: 31914


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
vectorizer = TextVectorization(
    max_tokens=MAX_FEATURES,
    output_sequence_length=OUTPUT_SEQUENCE_LENGTH,
    output_mode='int'
)
vectorizer.adapt(train_comments)

In [15]:
with open('vectorizer_assets.pkl', 'wb') as f:
    pickle.dump(vectorizer.get_weights(), f)

In [16]:
class_weights = {}
for i, col in enumerate(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']):
    weights = compute_class_weight('balanced', classes=np.array([0, 1]), y=train_labels[:, i])
    class_weights[i] = {0: weights[0], 1: weights[1]}

In [17]:
# Save class weights for Flask app
with open('class_weights.pkl', 'wb') as f:
    pickle.dump(class_weights, f)

In [18]:
def weighted_binary_crossentropy(y_true, y_pred):
    loss = 0
    for i in range(6):
        weight = tf.where(y_true[:, i] == 1, class_weights[i][1], class_weights[i][0])
        weight = tf.cast(weight, tf.float32)
        loss += tf.reduce_mean(weight * tf.keras.losses.binary_crossentropy(y_true[:, i], y_pred[:, i]))
    return loss / 6

In [19]:
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super().__init__(name=name, **kwargs)
        self.precision = Precision(thresholds=PREDICTION_THRESHOLD)
        self.recall = Recall(thresholds=PREDICTION_THRESHOLD)

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * ((p * r) / (p + r + tf.keras.backend.epsilon()))

    def reset_state(self):
        self.precision.reset_state()
        self.recall.reset_state()

In [ ]:
model = tf.keras.Sequential([
    vectorizer,
    Embedding(input_dim=MAX_FEATURES, output_dim=EMBEDDING_DIM, input_length=OUTPUT_SEQUENCE_LENGTH),
    Bidirectional(LSTM(LSTM_UNITS, activation='tanh')),  
    Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    Dropout(0.4),
    Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),  
    Dropout(0.4),
    Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    Dense(6, activation='sigmoid', dtype='float32')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.compile(
    loss=weighted_binary_crossentropy,
    optimizer='adam',
    metrics=['accuracy', Precision(thresholds=PREDICTION_THRESHOLD), Recall(thresholds=PREDICTION_THRESHOLD), F1Score()]   
)


In [22]:
model.build(input_shape=(None,))
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_vectorization              │ (None, 1800)           │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 1800, 64)       │     6,400,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 64)             │        24,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,499,846 (24.79 MB)

 Trainable params: 6,499,846 (24.79 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_comments, train_labels))
train_dataset = train_dataset.cache().shuffle(160000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)   

val_dataset = tf.data.Dataset.from_tensor_slices((val_comments, val_labels))
val_dataset = val_dataset.cache().batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [24]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('toxicity_model_checkpoint.keras', save_best_only=True, monitor='val_f1_score', mode='max')
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_f1_score', patience=3, mode='max')

In [25]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('toxicity_model_checkpoint.h5', save_best_only=True, monitor='val_f1_score', mode='max')
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_f1_score', patience=3, mode='max')

In [27]:
model.fit(
    train_dataset,
    epochs=10,
    validation_data=val_dataset,
    callbacks=[checkpoint, early_stopping]
)

Epoch 1/10
998/998 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step - accuracy: 0.8840 - f1_score: 0.2825 - loss: 0.8565 - precision: 0.2480 - recall: 0.3656

998/998 ━━━━━━━━━━━━━━━━━━━━ 155s 142ms/step - accuracy: 0.8841 - f1_score: 0.2827 - loss: 0.8559 - precision: 0.2482 - recall: 0.3658 - val_accuracy: 0.9943 - val_f1_score: 0.7154 - val_loss: 0.0844 - val_precision: 0.7451 - val_recall: 0.6879
Epoch 2/10
998/998 ━━━━━━━━━━━━━━━━━━━━ 139s 140ms/step - accuracy: 0.9919 - f1_score: 0.7238 - loss: 0.0802 - precision: 0.6985 - recall: 0.7511 - val_accuracy: 0.9943 - val_f1_score: 0.6485 - val_loss: 0.0981 - val_precision: 0.7112 - val_recall: 0.5959
Epoch 3/10
998/998 ━━━━━━━━━━━━━━━━━━━━ 141s 141ms/step - accuracy: 0.9931 - f1_score: 0.7324 - loss: 0.0746 - precision: 0.6955 - recall: 0.7738 - val_accuracy: 0.9943 - val_f1_score: 0.7104 - val_loss: 0.0746 - val_precision: 0.6605 - val_recall: 0.7686
Epoch 4/10
998/998 ━━━━━━━━━━━━━━━━━━━━ 139s 139ms/step - accuracy: 0.9909 - f1_score: 0.7619 - loss: 0.0629 - precision: 0.7168 - recall: 0.8131 - val_accuracy: 0.9943 - val_f1_score: 0.7080 - val_loss: 0.0711 - val_precision: 0.6441 - val_re

In [28]:
model.save('toxicity_model.h5')

print("Model and vectorizer saved successfully.")

Model and vectorizer saved successfully.


In [29]:
val_preds = (model.predict(val_dataset) > PREDICTION_THRESHOLD).astype(int)
macro_f1 = f1_score(val_labels, val_preds, average='macro')
print(f"Validation Macro F1 Score: {macro_f1}")
for i, col in enumerate(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']):
    f1 = f1_score(val_labels[:, i], val_preds[:, i])
    cm = confusion_matrix(val_labels[:, i], val_preds[:, i])
    print(f"{col} F1 Score: {f1}")
    print(f"{col} Confusion Matrix:\n{cm}")

print("Model, vectorizer, and class weights saved successfully.")

250/250 ━━━━━━━━━━━━━━━━━━━━ 12s 48ms/step
Validation Macro F1 Score: 0.4373173226452018
toxic F1 Score: 0.7733698130414957
toxic Confusion Matrix:
[[27879   973]
 [  518  2544]]
severe_toxic F1 Score: 0.40179573512906847
severe_toxic Confusion Matrix:
[[31202   392]
 [  141   179]]
obscene F1 Score: 0.7558375634517767
obscene Confusion Matrix:
[[29463   782]
 [  180  1489]]
threat F1 Score: 0.0
threat Confusion Matrix:
[[31812     0]
 [  102     0]]
insult F1 Score: 0.69290082424887
insult Confusion Matrix:
[[29456   900]
 [  255  1303]]
identity_hate F1 Score: 0.0
identity_hate Confusion Matrix:
[[31607     0]
 [  307     0]]
Model, vectorizer, and class weights saved successfully.


In [30]:
model.save('toxicity_model.keras')